In [ ]:
#@title **Create User**
#@markdown Enter Username and Password
 
import os
 
username = "user" #@param {type:"string"}
password = "root" #@param {type:"string"}
 
print("Creating User and Setting it up")
 
# Creation of user
os.system(f"useradd -m {username}")
 
# Add user to sudo group
os.system(f"adduser {username} sudo")
    
# Set password of user to 'root'
os.system(f"echo '{username}:{password}' | sudo chpasswd")
 
# Change default shell from sh to bash
os.system("sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd")
 
print("User Created and Configured")

Creating User and Setting it up
User Created and Configured


In [ ]:
from numba import jit, cuda
import numpy as np
# to measure exec time
from timeit import default_timer as timer
 
# normal function to run on cpu
def func(a):        
 for i in range(10000000):
  a[i]+= 1 
 
      
def func2(a):
 for i in range(10000000):
  a[i]+= 1
if __name__=="__main__":
 n = 10000000       
 a = np.ones(n, dtype = np.float64)
 b = np.ones(n, dtype = np.float32)
    
 start = timer()
 func(a)
 print("without GPU:", timer()-start) 
    
 start = timer()
 func2(a)
 print("with GPU:", timer()-start)

In [ ]:
import random, string, urllib.request, json, getpass
 
#Generate root password
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))
 
#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
 
#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
 
#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc
 
#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')
 
#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
authtoken = getpass.getpass()
 
#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
 
#Get public address and print connect command
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  (host, port) = data['tunnels'][0]['public_url'][6:].split(':')
  print(f'SSH command: ssh -p{port} root@{host}')
 
#Print root password
print(f'Root password: {password}')

In [ ]:
#@title **RDP**
#@markdown  It takes 4-5 minutes for installation
 
import os
import subprocess
 
#@markdown  Visit http://remotedesktop.google.com/headless and Copy the command after authentication
 
CRP = "DISPLAY= /opt/google/chrome-remote-desktop/start-host --code=\"4/0AY0e-g4kRhOersjy2dzOxhxne4XDgPSgYa35BGijoSgw0_lzpIdkjca4fQY1XgWqvKQocg\" --redirect-url=\"https://remotedesktop.google.com/_/oauthredirect\" --name=$(hostname)" #@param {type:"string"}
 
#@markdown Enter a pin more or equal to 6 digits
Pin = 123456 #@param {type: "integer"}
 
 
class CRD:
    def __init__(self):
        os.system("apt update")
        self.installCRD()
        self.installDesktopEnvironment()
        self.installGoogleChorme()
        self.finish()
 
    @staticmethod
    def installCRD():
        print("Installing Chrome Remote Desktop")
        subprocess.run(['wget', 'https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['dpkg', '--install', 'chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)
 
    @staticmethod
    def installDesktopEnvironment():
        print("Installing Desktop Environment")
        os.system("export DEBIAN_FRONTEND=noninteractive")
        os.system("apt install --assume-yes xfce4 desktop-base xfce4-terminal")
        os.system("bash -c 'echo \"exec /etc/X11/Xsession /usr/bin/xfce4-session\" > /etc/chrome-remote-desktop-session'")
        os.system("apt remove --assume-yes gnome-terminal")
        os.system("apt install --assume-yes xscreensaver")
        os.system("systemctl disable lightdm.service")
 
    @staticmethod
    def installGoogleChorme():
        print("Installing Google Chrome")
        subprocess.run(["wget", "https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(["dpkg", "--install", "google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)
 
    @staticmethod
    def finish():
        print("Finalizing")
        os.system(f"adduser {username} chrome-remote-desktop")
        command = f"{CRP} --pin={Pin}"
        os.system(f"su - {username} -c '{command}'")
        os.system("service chrome-remote-desktop start")
        print("Finished Succesfully")
 
 
try:
    if username:
        if CRP == "":
            print("Please enter authcode from the given link")
        elif len(str(Pin)) < 6:
            print("Enter a pin more or equal to 6 digits")
        else:
            CRD()
except NameError as e:
    print("username variable not found")
    print("Create a User First")

Installing Chrome Remote Desktop
Installing Desktop Environment


In [ ]:
#@title Package Installer { vertical-output: true }
run = False #@param {type:"boolean"}
#@markdown *Package management actions (gasp)*
action = "Install" #@param ["Install", "Check Installed", "Remove"] {allow-input: true}
 
package = "wget" #@param {type:"string"}
system = "apt" #@param ["apt", ""]
 
def install(package=package, system=system):
  if system == "apt":
    !apt --fix-broken install > /dev/null 2>&1
    !killall apt > /dev/null 2>&1
    !rm /var/lib/dpkg/lock-frontend
    !dpkg --configure -a > /dev/null 2>&1
 
    !apt-get  install -o Dpkg::Options::="--force-confold" --no-install-recommends -y $package
    
    !dpkg --configure -a > /dev/null 2>&1 
    !apt  update > /dev/null 2>&1
 
    !apt install $package > /dev/null 2>&1
 
def check_installed(package=package, system=system):
  if system == "apt":
    !apt list --installed | grep $package
 
def remove(package=package, system=system):
  if system == "apt":
    !apt remove $package
 
if run:
  if action == "Install":
    install()
  if action == "Check Installed":
    check_installed()
  if action == "Remove":
    remove()